In [5]:
import os
import json
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2 as cv
import glob
import torch
from lxml import etree
import tqdm
import numpy as np

In [ ]:
annotations_file="line_data/labels",
img_dir="line_data/images"
img_list=glob.glob(img_dir + "/*/*/*.png")

In [ ]:
len(img_list)

In [ ]:
label_dict={}
count=0
for file in tqdm.tqdm(label_dir):
    tree = ET.parse(source=os.path.join(label,file), parser=None)
    root = tree.getroot()
    count+=1
    for child in root.iter("line"):
        data = child.attrib
        label_dict[data["id"]] = data["text"]


In [ ]:
images=list()
for i in tqdm.tqdm(img_dir):
    images.append(cv.imread(i))

In [ ]:
def image_lbl(index):
    img=cv.imread(img_dir[index])
    img = 255 - img
    img_height, img_width = img.shape[0], img.shape[1]
    #rate = float(IMAGE_HEIGHT) / img.shape[0]
    #img = cv.resize(img, (int(img.shape[1]*rate)+1, IMAGE_HEIGHT), interpolation=cv.INTER_CUBIC)
    ratio = IMAGE_HEIGHT / img_height

    # Calculate the new width based on the ratio
    n_repeats = int(np.ceil(IMAGE_WIDTH/img_width))
    padded_image = np.concatenate([img]*n_repeats, axis=1)    
    padded_image = padded_image[:IMAGE_HEIGHT, :IMAGE_WIDTH]
    resized_img = cv.resize(padded_image, (IMAGE_WIDTH, desired_height))
    '''    print(padded_image.shape)
    plt.imshow(padded_image)
    plt.axis('off')
    plt.show()'''
    label=label_dict[os.path.basename(img_dir[index])[:-4]]
    return padded_image,label
    

In [ ]:
print(image_lbl(16)[1])
print(image_lbl(16)[0].shape)
plt.imshow(image_lbl(16)[0])
plt.axis('off')
plt.show()

In [ ]:
def read_image_single(file_name):
    img = cv.imread(file_name, 0)
    if img is None and os.path.exists(file_name):
        
        # image is present but corrupted
        return np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH)), 0

    rate = float(IMAGE_HEIGHT) / img.shape[0]
    img = cv.resize(img, (int(img.shape[1]*rate)+1, IMAGE_HEIGHT), interpolation=cv.INTER_CUBIC)
    img = img/255. # 0-255 -> 0-1

    img = 1. - img
    img_width = img.shape[-1]

    if img_width > IMAGE_WIDTH:
        print("coming here")
        outImg = img[:, :IMAGE_WIDTH]
        img_width = IMAGE_WIDTH
    else:
        
        outImg = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH), dtype='float32')
        outImg[:, :img_width] = img
        outImg = outImg.astype('float32')

    mean = 0.5
    std = 0.5
    outImgFinal = (outImg - mean) / std
    print(outImgFinal.shape)
    return outImgFinal, img_width

In [ ]:
img,_=read_image_single(file_name=img_dir[200])
print(label_dict[os.path.basename(img_dir[200])[:-4]])


In [ ]:
random_idxs = np.random.choice(len(img_dir), 10, replace = True)
data=[image_lbl(index) for index in random_idxs]

In [ ]:
import torchvision.transforms as transforms
from PIL import Image

def get_transform(grayscale=False, convert=True):

    transform_list = []
    if grayscale:
        transform_list.append(transforms.Grayscale(1))

    if convert:
        transform_list += [transforms.ToTensor()]
        if grayscale:
            transform_list += [transforms.Normalize((0.5,), (0.5,))]
        else:
            transform_list += [transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]

    return transforms.Compose(transform_list)

In [ ]:

# Read the image

# Get the original height and width
height, width = img.shape[:2]

# Define the desired output height
desired_height = 324

# Calculate the ratio of the new height to the old height
ratio = desired_height / height

# Calculate the new width based on the ratio
desired_width = int(width * ratio)

# Resize the image while preserving the aspect ratio
resized_img = cv.resize(img, (2270, desired_height))
# Save the output image
plt.imshow(resized_img)
plt.axis('off')
plt.show()

In [ ]:
assert len(json_path)==len(image_list)

In [ ]:
 
def imag_label(index):
# Reading the data inside the xml
# file to a variable under the name
# data
    IMG_WIDTH=350
    IMG_HEIGHT=450
    with open(json_path[index], 'r') as f:
        data = f.read()
    Bs_data = BeautifulSoup(data, "xml")
    b_unique = Bs_data.find('machine-printed-part')
    label=str(b_unique).replace("machine-printed-part","").replace("=","").replace("/","").replace("machine-print-line text","").replace('\n' , '').replace('<', '').replace('>', '').replace("/=","").replace('""','').replace("//","")
    print(label)
    print("reference image",image_list[index])
    img = cv.imread(image_list[index],0)
    print("image height and width",img.shape)
    if img is None and os.path.exists(image_list[index]):
            # image is present but corrupted
            return np.zeros((IMG_HEIGHT, IMG_WIDTH)), 0

    rate = float(IMG_HEIGHT) / img.shape[0]
    img = cv.resize(img, (int(img.shape[1]*rate)+1, IMG_HEIGHT), interpolation=cv.INTER_CUBIC)
    img = img/255. # 0-255 -> 0-1

    img = 1. - img
    img_width = img.shape[-1]

    if img_width > IMG_WIDTH:
        outImg = img[:, :IMG_WIDTH]
        img_width = IMG_WIDTH
    else:
        outImg = np.zeros((IMG_HEIGHT, IMG_WIDTH), dtype='float32')
        outImg[:, :img_width] = img
    outImg = outImg.astype('float32')

    mean = 0.5
    std = 0.5
    outImgFinal = (outImg - mean) / std
    plt.imshow(outImgFinal)
    plt.axis('off')
    plt.show()

In [ ]:
imag_label(index=0)

In [ ]:
import torch
import torch.nn as nn

# create a linear layer with input size 5376 and output size 1024
linear = nn.Linear(in_features=5376, out_features=1024)

# create a batch of tensors with shape (batch_size, 84, 64)
batch_tensor = torch.randn(( 84, 64))

# reshape the tensor to (batch_size, input_size)
input_tensor = batch_tensor.view(batch_tensor.size(0), -1)

# pass the input tensor to the linear layer



In [ ]:
batch=4
n_vocab=85
n_embedding=64

In [ ]:
linear = nn.Linear(64, 512)
embedding=nn.Embedding(n_vocab,n_embedding)

In [ ]:
Height=342
Width=2270

In [ ]:
embed=torch.rand(batch,n_vocab,n_embedding)
embed.shape

In [ ]:
xxx=embed.reshape(batch,-1)
xxx.shape

In [ ]:
xx_new=linear(embed) #4,83,512
xx_new.shape


In [ ]:
ts=xx_new.shape[1]
ts

In [ ]:
height_reps=Height
width_reps=Width//ts
(width_reps)

In [ ]:
out=torch.rand(batch,4096)

In [ ]:
tensor_list=list()
for i in range(ts):
    text = [xx_new[:, i:i + 1]] # b, text_max_len, 51281,81,512
    tmp = torch.cat(text * int(width_reps), dim=1) #batch,width_reps,512
    tensor_list.append(tmp) #ts batch_size,width_reps,512
print(len(tensor_list))    

In [ ]:
len(tensor_list)  
"""[-2,-1]
                height and width"""

In [ ]:
tokens = {'GO_TOKEN': 0, 'END_TOKEN': 1, 'PAD_TOKEN': 2}

In [ ]:
2270%ts

In [ ]:
padding_reps=2270%ts #10
if padding_reps:
    print("welcome here i dont knwo but welocme")
    char_embeding=embedding(torch.full((1,1),tokens["PAD_TOKEN"],dtype=torch.long)) #(1,1,n_embeding)
    char_embeding=linear(char_embeding) #(1,1,512)
    padding=char_embeding.repeat(batch,padding_reps,1) ##torch.Size([4, 10, 512])
    tensor_list.append(padding)
print(len(tensor_list))

In [ ]:
tensor_list[0].size()

In [ ]:
"""batch,text_max,width_reps,+padding,512
   bach, ts+1*padding,512

"""
res=torch.cat(tensor_list, dim=1)


In [ ]:
res.shape

In [ ]:
res = res.permute(0, 2, 1).unsqueeze(2) # b, 512, 1, text_max_len * width_reps + padding_reps
res.shape

In [ ]:
final_res = torch.cat([res] * height_reps, dim=2)

In [ ]:
final_res.shape

In [2]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset( ):
    def __init__(self, annotations_file="line_data/labels", img_dir="line_data/images", transform=None, target_transform=None):
        self.label_dict={}
        self.IMAGE_HEIGHT=100
        self.IMAGE_WIDTH=500
        self.img_path = img_dir
        #self.transform = get_transform(grayscale=True)
        self.num_example=10
        self.label_path=annotations_file
        self.label_dir = os.listdir(self.label_path)
        self.img_dir=(glob.glob(self.img_path+"/*/*/*.png"))
        del(self.label_dir[1230])
        for file in tqdm.tqdm(self.label_dir):
            tree = ET.parse(source=os.path.join(self.label_path,file), parser=None)
            root = tree.getroot()
            for child in root.iter("line"):
                data = child.attrib
                self.label_dict[data["id"]] = data["text"]
    def image_lbl(self,index):
        img=cv.imread(self.img_dir[index])
        img = 255 - img
        img_height, img_width = img.shape[0], img.shape[1]
        n_repeats = int(np.ceil(self.IMAGE_WIDTH/img_width))
        padded_image = np.concatenate([img]*n_repeats, axis=1)
        padded_image = padded_image[:self.IMAGE_HEIGHT, :self.IMAGE_WIDTH]
        resized_img = cv.resize(padded_image, (self.IMAGE_WIDTH, self.IMAGE_HEIGHT))
        label=self.label_dict[os.path.basename(self.img_dir[index])[:-4]]
        return resized_img,label
        ''' print(padded_image.shape)
        plt.imshow(padded_image)
        plt.axis('off')
        plt.show()
        print()'''
    def __len__(self):
        return len(self.label_dict)

    def __getitem__(self, idx):
        Images=list()
        Labels=list()
        data_set={}
        random_idxs = np.random.choice(len(self.img_dir), 5, replace = True)
        
        for index in random_idxs:
            Img,lbl=self.image_lbl(index)
            Images.append(torch.tensor(Img).float())
            #Images.append(torch.nn.functional.pad(tensor, (0, self.IMAGE_WIDTH - tensor.size(2), 0, self.IMAGE_HEIGHT - tensor.size(1))))
            Labels.append(lbl)
        concate_image=torch.cat(Images,0)   
        '''if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)'''
        return Images,Labels

In [3]:
class ResNET(nn.Module):
    # see https://pytorch.org/docs/0.4.0/_modules/torchvision/models/resnet.html
    def __init__(self, inplanes, planes, stride=1):
        super(ResNET, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.relu(out)
        return out

NameError: name 'nn' is not defined

In [ ]:
class ResBlocks(nn.Module):
    def __init__(self, num_blocks, in_dim,out_dim):
        super(ResBlocks, self).__init__()
        self.model = []
        for i in range(num_blocks):
            self.model += [ResNET(in_dim,
                                    out_dim,
                                    )]
        self.model = nn.Sequential(*self.model)

    def forward(self, x):
        return self.model(x)

In [ ]:
nets=ResBlocks(6,3,512)
nets

In [ ]:
data=CustomImageDataset()


In [ ]:
len(data)

In [4]:
data=CustomImageDataset()


from torch.utils.data import DataLoader

dataset = DataLoader(data, batch_size=5, shuffle=True)


NameError: name 'glob' is not defined

In [ ]:
dataset

In [ ]:
1539/5

In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
print(len(train_dataset),len(test_dataset))

In [ ]:
for i in train_data_loader:
    print(i)

In [ ]:
train_data_loader=DataLoader(train_dataset,batch_size=2,shuffle=True)
test_data_loader=DataLoader(test_dataset,batch_size=2,shuffle=True)
print(len(train_data_loader),len(test_data_loader))

In [ ]:
(len(dataset)*0.2)

In [ ]:
count=0
for i in tqdm.tqdm(train_dataloader):
    count+=1
print(count)

In [ ]:
2, 3, 2270, 342
img[i].permute(0,3,2,1).shape

In [ ]:
torch.rand(2, 342, 2270, 3)

In [ ]:
pytorch_total_params = sum(p.numel() for p in nets.parameters())
pytorch_total_params

In [ ]:
pytorch_total_params = sum(p.numel() for p in nets.parameters() if p.requires_grad)
pytorch_total_params

In [ ]:
net=ResNet(in_channels=3, output=512, block_sizes=2)
pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
pytorch_total_params
nets

In [79]:
import torch
import torch.nn as nn

class Head(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        
        # Q, K, V weight matrices for each head
        self.wq = nn.Linear(d_model, d_model, bias=False)
        self.wk = nn.Linear(d_model, d_model, bias=False)
        self.wv = nn.Linear(d_model, d_model, bias=False)
        self.scale = 1.0 / (d_model ** 0.5)

        # Output projection matrix
        self.proj = nn.Linear(d_model, d_model, bias=False)
        
    def forward(self, x):
        # x shape: [batch_size, num_channels, image_height, image_width]
        batch,channel,high,width=x.shape
        # Reshape input to [batch_size, num_channels*image_height, image_width]
        x = x.view(x.size(0), -1, x.size(1))
        
        # Compute Q, K, V matrices for each head
        q = self.wq(x)  # q shape: [batch_size, num_channels*image_height, d_model]
        k = self.wk(x)  # k shape: [batch_size, num_channels*image_height, d_model]
        v = self.wv(x)  # v shape: [batch_size, num_channels*image_height, d_model]
        weights = torch.matmul(q, k.transpose(-2, -1))
        weights = weights * self.scale
        weights = nn.functional.softmax(weights, dim=-1)

        # Apply attention weights to values
        output = torch.matmul(weights,v )
        return output.view(batch,channel,high,width)


In [88]:
x.view(x.size(0), -1, x.size(1)).shape

torch.Size([2, 200, 32])

In [81]:
import torch

# Instantiate MultiheadAttention layer with d_model=32 and num_heads=4
mha = Head(d_model=32, num_heads=4)

# Create input tensor
x = torch.randn(2, 32, 20, 10)

# Pass input tensor through MultiheadAttention layer
out = mha(x)

# Print output tensor shape
print(out.shape)


torch.Size([2, 32, 20, 10])


In [96]:
class MultiHeadAttention(nn.Module):
  "Multiple heads of the self_attention in parallel"
  def __init__(self,num_heads,dropout):
    super().__init__()
    self.heads=nn.ModuleList([Head(d_model=32, num_heads=num_heads) for _ in range(num_heads)])
    self.dropout=nn.Dropout(dropout)

  def forward(self,x):
    out= torch.cat([h(x) for h in self.heads])

    out=self.dropout (out)
    print(out.shape)
    return out
  


In [97]:
model=MultiHeadAttention(4,0.2)

In [104]:
out=model(x)

torch.Size([8, 32, 20, 10])


In [123]:
out.unsqueeze(0).shape

torch.Size([1, 8, 32, 20, 10])

In [136]:
b = x.repeat(out.size(0)//2, 1, 1, 1)
torch.prod(torch.tensor((2,3,4)))

tensor(24)

In [138]:
import torch
import torch.nn as nn

# Define the input tensor
input_tensor = torch.randn(8, 32, 20, 40)

# Define the linear layer with the same input and output size
linear_layer = nn.Linear(40, 40)

# Flatten the tensor to make it 2D for the linear layer
flatten_tensor = input_tensor.view(-1, 40)

# Apply the linear transformation
output_tensor = linear_layer(flatten_tensor)

# Reshape the output tensor to the original shape
output_tensor = output_tensor.view(8, 32, 20, 40)

# Print the output tensor shape
print(output_tensor.shape)


torch.Size([8, 32, 20, 40])


40